### Spam Detection

In this problem, we will be constructing a crude spam detector. When we receive an e-mail, it can be divided into one of two types: ham (useful mail, label $-1$) and spam (junk mail, label $+1$).  

We will be designing a spam detector by applying some of the classification techniques such as 
- Naive Bayes
- Logistic Regression
- Support Vector Machines

to a batch of emails used to train and test [SpamAssassin](http://spamassassin.apache.org/), a leading anti-spam software package. 

Let the *vocabulary* of a dataset be a list of all terms occuring in a data set. So, for example, a vocabulary could be ["cat","dog","chupacabra", "aerospace", ...]. 

Our features will be based only the frequencies of terms in our vocabulary occuring in the e-mails (such an approach is called a *bag of words* approach, since we ignore the positions of the terms in the emails). The $j$-th feature is the number of times term $j$ in the vocabulary occurs in the email.

We will use the following classifiers in this problem:
- sklearn.naive_bayes.BernoulliNB (Naive Bayes Classifier with Bernoulli Model)
- sklearn.naive_bayes.MultinomialNB (Naive Bayes Classifier with Multinomial Model)
- sklearn.svm.LinearSVC (Linear Support Vector Machine)
- sklearn.linear_model.LogisticRegression (Logistic Regression)
- sklearn.neighbors.KNeighborsClassifier (1-Nearest Neighbor Classifier)

In the context of the Bernoulli Model for Naive Bayes, scikit-learn will binarize the features by interpretting the $j$-th feature to be $1$ if the $j$-th term in the vocabulary occurs in the email and $0$ otherwise. This is a categorical Naive Bayes model, with binary features. The multinomial model operates directly on the frequencies of terms in the vocabulary.

A sample Ham email is:

    From nic@starflung.com  Mon Jun 24 17:06:54 2002
    Return-Path: 7910726.0.27May2002215326@mp.opensrs.net
    Delivery-Date: Tue May 28 02:53:28 2002
    Received: from mp.opensrs.net (mp.opensrs.net [216.40.33.45]) by
        dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g4S1rSe14718 for
        <zzz@spamassassin.taint.org>; Tue, 28 May 2002 02:53:28 +0100
    Received: (from popensrs@localhost) by mp.opensrs.net (8.9.3/8.9.3) id
        VAA04361; Mon, 27 May 2002 21:53:26 -0400
    Message-Id: <7910726.0.27May2002215326@mp.opensrs.net>
    Date: Mon, 27 May 2002 21:53:26 -0500 (EST)
    From: "Starflung NIC" <nic@starflung.com>
    To: <zzz@spamassassin.taint.org>
    Subject: Automated 30 day renewal reminder 2002-05-27
    X-Keywords: 

    The following domains that are registered as belonging
    to you are due to expire within the next 60 days. If
    you would like to renew them, please contact
    nic@starflung.com; otherwise they will be deactivated
    and may be registered by another.


    Domain Name, Expiry Date
    nutmegclothing.com, 2002-06-26
    
    
A sample Spam email is: 

    From jjj@mymail.dk  Fri Aug 23 11:03:31 2002
    Return-Path: <jjj@mymail.dk>
    Delivered-To: zzzz@localhost.example.com
    Received: from localhost (localhost [127.0.0.1])
        by phobos.labs.example.com (Postfix) with ESMTP id 478B54415C
        for <zzzz@localhost>; Fri, 23 Aug 2002 06:02:57 -0400 (EDT)
    Received: from mail.webnote.net [193.120.211.219]
        by localhost with POP3 (fetchmail-5.9.0)
        for zzzz@localhost (single-drop); Fri, 23 Aug 2002 11:02:57 +0100 (IST)
    Received: from smtp.easydns.com (smtp.easydns.com [205.210.42.30])
        by webnote.net (8.9.3/8.9.3) with ESMTP id IAA08912;
        Fri, 23 Aug 2002 08:13:36 +0100
    From: jjj@mymail.dk
    Received: from mymail.dk (unknown [61.97.34.233])
        by smtp.easydns.com (Postfix) with SMTP
        id 7484A2F85C; Fri, 23 Aug 2002 03:13:31 -0400 (EDT)
    Reply-To: <jjj@mymail.dk>
    Message-ID: <008c61d64eed$6184e5d5$4bc22de3@udnugg>
    To: bbr_hooten@yahoo.com
    Subject: HELP WANTED.  WORK FROM HOME REPS.
    MiME-Version: 1.0
    Content-Type: text/plain;
        charset="iso-8859-1"
    X-Priority: 3 (Normal)
    X-MSMail-Priority: Normal
    X-Mailer: Microsoft Outlook, Build 10.0.2616
    Importance: Normal
    Date: Fri, 23 Aug 2002 03:13:31 -0400 (EDT)
    Content-Transfer-Encoding: 8bit

    Help wanted.  We are a 14 year old fortune 500 company, that is
    growing at a tremendous rate.  We are looking for individuals who
    want to work from home.

    This is an opportunity to make an excellent income.  No experience
    is required.  We will train you.

    So if you are looking to be employed from home with a career that has
    vast opportunities, then go:

    http://www.basetel.com/wealthnow

    We are looking for energetic and self motivated people.  If that is you
    than click on the link and fill out the form, and one of our
    employement specialist will contact you.

    To be removed from our link simple go to:

    http://www.basetel.com/remove.html


    1349lmrd5-948HyhJ3622xXiM0-290VZdq6044fFvN0-799hUsU07l50



First, we will load the data. Our dataset has a bit over 9000 emails, with about 25% of them being spam. We will use 50% of them as a training set, 25% of them as a validation set and 25% of them as a test set. 

In [1]:
%pylab inline
import numpy as np
from sklearn import neighbors
from sklearn import svm
from sklearn import model_selection
from numpy import genfromtxt
from sklearn.preprocessing import MinMaxScaler
import glob

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Get list of emails
spamfiles=glob.glob('./Data/Spam/*')
hamfiles=glob.glob('./Data/Ham/*')
type(hamfiles)

list

In [4]:
# First, we will split the files into the training, validation and test sets.

np.random.seed(seed=222017) # seed the RNG for repeatability

fnames=np.asarray(spamfiles+hamfiles)
nfiles=fnames.size
labels=np.ones(nfiles)
labels[len(spamfiles):]=-1

# Randomly permute the files we have
idx=np.random.permutation(nfiles)
fnames=fnames[idx]
labels=labels[idx]

#Split the file names into which set they belong to
tname=fnames[:int(nfiles/2)]
trainlabels=labels[:int(nfiles/2)]
vname=fnames[int(nfiles/2):int(nfiles*3/4)]
vallabels=labels[int(nfiles/2):int(nfiles*3/4)]
tename=fnames[int(3/4*nfiles):]
testlabels=labels[int(3/4*nfiles):]

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

# Get our Bag of Words Features from the data
bow = CountVectorizer(input='filename',encoding='iso-8859-1',binary=False)
traindata=bow.fit_transform(tname)
valdata=bow.transform(vname)
testdata=bow.transform(tename)
type(fnames)

numpy.ndarray

The $100$ most and least common terms in the vocabulary are:

In [6]:
counts=np.reshape(np.asarray(np.argsort(traindata.sum(axis=0))),-1)
vocab=np.reshape(np.asarray(bow.get_feature_names()),-1)
print ("100 most common terms: " , ','.join(str(s) for s in vocab[counts[-100:]]), "\n")
print ("100 least common terms: " , ','.join(str(s) for s in vocab[counts[:100]]))

100 most common terms:  as,have,not,slashnull,dogma,request,thu,ist,mail,cnet,are,lists,wed,jmason,or,exmh,freshrpms,html,mailman,mon,00,12,date,message,ie,text,users,bgcolor,postfix,09,type,align,rpm,linux,arial,22,version,be,your,taint,mailto,admin,content,sourceforge,table,on,20,color,border,jm,aug,gif,example,this,127,href,img,face,src,10,subject,nbsp,that,it,sep,0100,height,spamassassin,esmtp,is,xent,fork,size,you,in,www,tr,br,list,11,width,received,of,and,localhost,id,org,by,with,net,for,td,http,font,3d,2002,from,the,to,com 

100 least common terms:  bjkwasvqaww7aidwbawacabaazqbxthxxtuhfzraa9cab4dabujgccrqcnxgasdgbe4abaudbviw,di09bpkfnyor9plfsy2k1x0rrjfrd3lyxe1pf9,painfree,di4ogzxbde4xbwtydpn3gowojdjjakl3teark4n2ezk4tu5wziny,di79lua5r2hzsc1wycoous63kg026atmbci2,di9f,diagn,paictkoa9mibhkigds8p6vm,pahdtw85r7jtuzknjeqccngoqyuk5cqub,pah,diagrammatic,pagetype,pagers,pagereq,dial12,painfull,dialcom,pageid,page1_11,page1_10,page1_09,page1_08,page1_07,dialoguez,dialpool,page1_06,page1_05

We will have our training data in `traindata` (with labels in `trainlabels`), validation data in `valdata` (with labels in `vallabels`) and test data in `testdata` (with labels in `testlabels`). The data is stored as a sparse scipy matrix (`scipy.sparse.csr.csr_matrix`), since we have a decent number of features (~ 100k), most of which are zero (~ 0.2% are non-zero), this allows storing the data in a few megabytes. Directly storing it as a numpy array (as we did in lab 1) would take around 8 gigabytes. Working with sparse data can make many algorithms run faster and use less storage. 

We train each of the following classifiers :
- sklearn.naive_bayes.BernoulliNB (Naive Bayes Classifier with Bernoulli Model)
- sklearn.naive_bayes.MultinomialNB (Naive Bayes Classifier with Multinomial Model)
- sklearn.svm.LinearSVC (Linear Support Vector Machine)
- sklearn.linear_model.LogisticRegression (Logistic Regression)
- sklearn.neighbors.KNeighborsClassifier (as a 1-Nearest Neighbor Classifier)
on the training data in `traindata` with corresponding labels `trainlabels`.

For each classifier, we report:
- Time it took to fit the classifier (i.e. call the .fit method)  
- Training Error 

In [3]:
from sklearn.naive_bayes import BernoulliNB 
from sklearn.naive_bayes import MultinomialNB 
from sklearn.svm import LinearSVC 
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.svm import SVC
import time
from sklearn.metrics import zero_one_loss

In [6]:

clfs=[BernoulliNB,MultinomialNB,LinearSVC,LogisticRegression,KNeighborsClassifier]
trained_objects=dict()
# clfs=["BernoulliNB","MultinomialNB"]
for x in clfs:
    
    if (x.__name__ == "KNeighborsClassifier"): #setting the KNeighborsClassifier k=1
        obj=x(n_neighbors=1)
    elif(x.__name__ == "LinearSVC"):   # setting dual = false or else the calculation fails to converge
        obj=x(dual=False)
    elif(x.__name__ == "LogisticRegression"): # increasing max_iter or the calculation fails to converge 
        obj=x(max_iter=7600,solver='lbfgs')
    else:
        obj=x()
        
    trained_objects[x.__name__] = obj   # associating variable names with classifier object
    srt_tm = time.time()  #starting the time
    obj.fit(traindata, trainlabels) #training the classfiers
    t_taken = time.time() - srt_tm #stopping the time
    print("Time taken in ",x.__name__,"Classifier %2.3f" %t_taken,"secs")
    print("Error in",x.__name__,"Classifier %2.3f" %(zero_one_loss(trainlabels,obj.predict(traindata))*100),"%")
    print("----------------------------------------------")
    

Time taken in  BernoulliNB Classifier 0.100 secs
Error in BernoulliNB Classifier 5.285 %
----------------------------------------------
Time taken in  MultinomialNB Classifier 0.021 secs
Error in MultinomialNB Classifier 1.861 %
----------------------------------------------
Time taken in  LinearSVC Classifier 5.709 secs
Error in LinearSVC Classifier 0.000 %
----------------------------------------------
Time taken in  LogisticRegression Classifier 10.830 secs
Error in LogisticRegression Classifier 0.000 %
----------------------------------------------
Time taken in  KNeighborsClassifier Classifier 0.005 secs
Error in KNeighborsClassifier Classifier 0.000 %
----------------------------------------------


Both Linear SVM and Logistic regression have zero training errors this is because data is linearly separable, and the classifiers just have to predict whether the email is spam or not (multinomial distribution with the 0-1 loss function).

Now we run each of the classifiers on the validation data:
- sklearn.naive_bayes.BernoulliNB (Naive Bayes Classifier with Bernoulli Model)
- sklearn.naive_bayes.MultinomialNB (Naive Bayes Classifier with Multinomial Model)
- sklearn.svm.LinearSVC (Linear Support Vector Machine)
- sklearn.linear_model.LogisticRegression (Logistic Regression)
- sklearn.neighbors.KNeighborsClassifier (as a 1-Nearest Neighbor Classifier)
on the training data in `traindata` with corresponding labels `trainlabels`.

For each classifier, we do the following:
- Store the labels it predicted as \_\_vallabels, where \_\_ is NB,MB,SVM,LR,NN respectively.
- Time it took to run the classifier on the data
- Validation Error

In [7]:
prefix=["NB_vallabels","MB_vallabels","SVM_vallabels","LR_vallabels","NN_vallabels"]
mapped_names=dict()
for x in trained_objects:
    srt_tm = time.time() #starting the time
    val_labels = trained_objects[x].predict(valdata)  # predicting the labels usign the classifires trained before
    t_taken = time.time() - srt_tm #stoping the time
    tracker = prefix[list(trained_objects.keys()).index(x)]
    locals()[tracker] = val_labels #associating the labels of different classifiers according to the variable names in the question
    mapped_names[tracker] = x
    print("Error in",x,"Classifier %2.3f" %(zero_one_loss(vallabels,val_labels)*100),"%")
    print("Time taken in ",x,"Classifier %2.3f" %t_taken,"secs")
    print("----------------------------------------------")   

Error in BernoulliNB Classifier 9.799 %
Time taken in  BernoulliNB Classifier 0.025 secs
----------------------------------------------
Error in MultinomialNB Classifier 3.594 %
Time taken in  MultinomialNB Classifier 0.014 secs
----------------------------------------------
Error in LinearSVC Classifier 1.455 %
Time taken in  LinearSVC Classifier 0.007 secs
----------------------------------------------
Error in LogisticRegression Classifier 1.369 %
Time taken in  LogisticRegression Classifier 0.007 secs
----------------------------------------------
Error in KNeighborsClassifier Classifier 2.268 %
Time taken in  KNeighborsClassifier Classifier 3.391 secs
----------------------------------------------


Let us take a more nuanced look at the type of errors made on a data set. The following function calculates a confusion matrix and some statistics.

In [8]:
def ConfMatr(truelabels,estimatedlabels,classifiername):
    # classifiername is a string, such as 'Naive Bayes (Bernoulli)'
    cm=np.zeros((2,2))
    cm[0,0]=np.sum(np.logical_and(truelabels==1,estimatedlabels==1)) # True Positives
    cm[0,1]=np.sum(np.logical_and(truelabels==-1,estimatedlabels==1)) # False Positive
    cm[1,0]=np.sum(np.logical_and(truelabels==1,estimatedlabels==-1)) # False Negative
    cm[1,1]=np.sum(np.logical_and(truelabels==-1,estimatedlabels==-1)) # True Negatives
    print ("Classifier Name: %s"% classifiername )
    print ("True Positives:", cm[0,0], "False Positive:", cm[0,1])
    print ("False Negative:", cm[1,0], "True Negatives:", cm[1,1])
    print ("True Positive Rate : ", cm[0,0]/np.sum(truelabels==1))
    print ("False Positive Rate: ", cm[0,1]/np.sum(truelabels==-1))
    print ("---")

We now run ConfMatr using the validation labels and their estimates for all the classifiers we've used in this problem.

In [9]:
for x in mapped_names:
    ConfMatr(vallabels, locals()[x], mapped_names[x])

Classifier Name: BernoulliNB
True Positives: 394.0 False Positive: 21.0
False Negative: 208.0 True Negatives: 1714.0
True Positive Rate :  0.654485049833887
False Positive Rate:  0.012103746397694525
---
Classifier Name: MultinomialNB
True Positives: 527.0 False Positive: 9.0
False Negative: 75.0 True Negatives: 1726.0
True Positive Rate :  0.8754152823920266
False Positive Rate:  0.005187319884726225
---
Classifier Name: LinearSVC
True Positives: 593.0 False Positive: 25.0
False Negative: 9.0 True Negatives: 1710.0
True Positive Rate :  0.9850498338870431
False Positive Rate:  0.01440922190201729
---
Classifier Name: LogisticRegression
True Positives: 591.0 False Positive: 21.0
False Negative: 11.0 True Negatives: 1714.0
True Positive Rate :  0.9817275747508306
False Positive Rate:  0.012103746397694525
---
Classifier Name: KNeighborsClassifier
True Positives: 577.0 False Positive: 28.0
False Negative: 25.0 True Negatives: 1707.0
True Positive Rate :  0.9584717607973422
False Positive

The True positive rate is the fraction of emails that were spam that the classifier also correctly classified them as spam. The false-positive rate is the fraction of emails that were not spam but the classifier classified them as spam. The true positive rate should be high (close to 1 meaning spam classified as spam) while the false positive rate should be low (close to 0 meaning we should not classify ham as spam) 

Based on the results of the classfiers on the validation set, we will choose Linear Support Vector Machine classifier as it has the highest true positive rates and one of the lowest false positive rates. We think this is an appropriate choice since it filters the most amount of spam emails. In a university setting the amount of spam must be really high so choosing the best classifier (based on true positive rate) is a good choice while making some tradeoff with the false positive rate. I will keep updating my training set as the users at the university mark emails as spam. This will help the classifier train on larger training set.
______________________________________________________________________________________________

We could add additional features to the SVM classifier based on the IP address of the sender, emails from a list of trusted email domains or if the email address are in the user's address book, email addressed to the recipient's name and not his or her email address. I will continuously update the training data if the user marks an email as spam or marks spam as ham. We will also check if the email has an attachment or not and the extension of the attachment(.pdf is safe while a .exe is not). Filtering everyone's email jointly may not be a good idea as it may slow down the process of classification however I will actively scan the number of recipients on the email, spam emails are generally sent to a lot of recipients.
    

Here we run the classifier we selected in the previous part of the problem on the test data, and display test error and output of ConfMatr. we also comment on the true/false positive rate and error as compared to that on the validation set.

In [10]:
test_labels = trained_objects["LinearSVC"].predict(testdata)
print("Error in",'LinearSVC',"Classifier %2.3f" %(zero_one_loss(testlabels,test_labels)*100),"%")
ConfMatr(testlabels, test_labels, "LinearSVC")

Error in LinearSVC Classifier 1.069 %
Classifier Name: LinearSVC
True Positives: 614.0 False Positive: 19.0
False Negative: 6.0 True Negatives: 1699.0
True Positive Rate :  0.9903225806451613
False Positive Rate:  0.011059371362048894
---


We note that the TPR was higher and FRP was lower on the test data than on the validation data. The error on the test data was also lower (by around 0.5%) as compared to the validation data. We also observe a reduction in the number of false-negative (spam incorrectly classified as ham).

### Detecting Cancer with SVMs and Logistic Regression 

We consider the [Breast Cancer Wisconsin Data Set](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29) from 
W.N. Street, W.H. Wolberg and O.L. Mangasarian. Nuclear feature extraction for breast tumor diagnosis. IS&T/SPIE 1993 International Symposium on Electronic Imaging: Science and Technology, volume 1905, pages 861-870, San Jose, CA, 1993. 

The authors diagnosed people by characterizing 3 cell nuclei per person extracted from the breast (pictures [here](http://web.archive.org/web/19970225174429/http://www.cs.wisc.edu/~street/images/)), each with 10 features (for a 30-dimensional feature space):

1. radius (mean of distances from center to points on the perimeter) 

2. texture (standard deviation of gray-scale values) 

3. perimeter 

4. area 

5. smoothness (local variation in radius lengths) 

6. compactness (perimeter^2 / area - 1.0) 

7. concavity (severity of concave portions of the contour) 

8. concave points (number of concave portions of the contour) 

9. symmetry 

10. fractal dimension ("coastline approximation" - 1)

and classified the sample into one of two classes: Malignant ($+1$) or Benign ($-1$). We will be attempting to classify if a sample is Malignant or Benign using Support Vector Machines, as well as Logistic Regression. Since we don't have all that much data, we will use 10-fold cross-validation to tune our parameters for our SVMs and Logistic Regression. We use 90% of the data for training, and 10% for testing.

We will be experimenting with SVMs using Gaussian RBF kernels through sklearn.svm.SVC, linear SVMs through sklearn.svm.LinearSVC, and sklearn.linear_model.LogisticRegression for Logistic Regression. 

Our model selection will be done with cross-validation via sklearn.model_selection's cross_val_score. This returns the accuracy for each fold, i.e. the fraction of samples classified correctly. Thus, the cross-validation error is simply 1-mean(cross_val_score).

First, we load the data. We will use scikit-learn's train test split function to split the data. The data is also scaled.

In [4]:
from sklearn.model_selection import train_test_split

cancer = genfromtxt('Data/wdbc.csv', delimiter=',')

np.random.seed(seed=282017) # seed the RNG for repeatability
idx=np.random.permutation(cancer.shape[0])
cancer=cancer[idx]

cancer_features=cancer[:,1:]
cancer_labels=cancer[:,0]

#The training data is in data_train with labels label_train. 
# The test data is in data_test with labels label_test.
data_train, data_test, label_train, label_test = train_test_split(cancer_features,cancer_labels,test_size=0.1,random_state=292017)

# Rescale the training data and scale the test data correspondingly
scaler=MinMaxScaler(feature_range=(-1,1))
data_train=scaler.fit_transform(data_train) #Note that the scaling is determined solely via the training data!
data_test=scaler.transform(data_test)

The soft margin linear SVM is tuned based on a parameter $C$, which controls how much points can be violating the margin. 

We use cross-validation to select a value of $C$ for a linear SVM (sklearn.svm.LinearSVC) by varying $C$ from $2^{-5},2^{-4},\ldots,2^{15}$. 

In [52]:
from sklearn.model_selection import cross_val_score
nos=range(-5,16,1)
C_data=[2**i for i in nos]
errors_C=[]
for x in C_data:
    clf = LinearSVC(dual=False,C=x)
    scores = cross_val_score(clf, data_train, label_train, cv=10)
    errors_C.append((1-(scores.mean()))*100)
# print(errors_C)
min_pos=np.argmin(np.asarray(errors_C))
print("Minimum error is ",errors_C[min_pos])
print("Postion of the min. error in C vector",min_pos)
print("Value of C=",C_data[min_pos],"=","2^-3")

Minimum error is  2.714932126696823
Postion of the min. error in C vector 2
Value of C= 0.125 = 2^-3


<b>The value of C=2^-3 should be choosen since it gives the minimum error cross-validation error which is about 2.71%

We will now experiment with using kernels in an SVM, particularly the Gaussian RBF kernel (in sklearn.svm.SVC). The SVM has two parameters to tune in this case: $C$ (as before), and $\gamma$, which is a parameter in the RBF. 

We use cross-validation to select parameters $(C,\gamma)$ by searching varying $(C,\gamma)$ over $C=2^{-5},2^{-4},\ldots,2^{15}$ and $\gamma=2^{-15},\ldots,2^{3}$ [So, we will try about 400 parameter choices]. Out of these, we choose $(C,\gamma)$ parameters that will give us min. cross-validation error.

We are using a fairly coarse grid for this problem, but one could use a finer grid once the rough range of good parameters is known (rather than starting with a fine grid, which would waste a lot of time). <b></b>

In [64]:

g_nos=range(-15,3,1)
G_data=[2**i for i in g_nos]
error_CG=np.zeros((len(C_data),len(g_nos)))
for i in range(0,len(C_data)):
    for j in range(0,len(G_data)):
        clf2 = SVC(C=C_data[i],gamma=G_data[j])
        scores = cross_val_score(clf2, data_train, label_train, cv=10)
        error_CG[i,j]=(1-scores.mean())*100


In [76]:
ind = np.unravel_index(np.argmin(error_CG, axis=None), error_CG.shape)
print(ind)
print("The min error is",error_CG[ind])
print("The value of C for min error is",C_data[ind[0]])
print("The value of Gamma for min is",G_data[ind[1]])

(8, 12)
The min error is 1.9457013574660564
The value of C for min error is 8
The value of Gamma for min is 0.125


<b>We should choose C=8=2^3 and Gamma=0.125=2^-3 as parameters for our classfiers as these give the minimum error which is 1.945%

We know that Logistic Regression normally has a regularizer parameter to promote stability. Scikit-learn calls this parameter $C$. We use cross-validation to select a value of $C$ for logistic regression (sklearn.linear_model.LogisticRegression) by varying $C$ from $2^{-14},2^{-4},\ldots,2^{14}$.

In [164]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
nos_new=range(-14,14,1)
C_new=[2**i for i in nos_new]
lgr_error=[]
for x in C_new:
    lgr = LogisticRegression(max_iter=5000,solver='lbfgs',C=x)
    scores = cross_val_score(lgr, data_train, label_train, cv=10)     
    lgr_error.append((1-(scores.mean()))*100)    

In [165]:
minerr=np.argmin(np.asarray(lgr_error))
print("The minimum error is",min(lgr_error))
print("The C value for minimum error is",C_new[minerr])


The minimum error is 2.1342383107088914
The C value for minimum error is 1


<b>The value of C that should be choose is C=1 = 2^0  which has 2.1342% cross validation error

<b> Based on our observations on the cross-validation errors so far for the optimum C and Gamma values: <br>
    Linear SVM - 2.71493%  <br>
    SVM + Gaussian RBF- 1.9457% <br>
    Logistic Regression-2.13423 % <br>
I would choose the Logistic classifier since its cross-validation error is one of the lowest and computationally takes much less time compared to SVM + Gaussian RBF which needs to optimize two parameters first. The logistic regression classifier error is just ~0.2 percent higher than SVM + Gaussian RBF and has a much lower computational cost. Considering that the application is predicting malignancy we have the option of further testing (biopsy) if our predictions are inaccurate and so lower computational cost (which lowers the price of the test) is preferred.